# Predicción Estructural de Proteínas

 Esta red intenta predecir  la estructura secundaria de una proteína  a partir de su secuencia  de aminoácidos, busca encontrar patrones asociados  a orientaciones tridimiencionales definidas dadas por los cuaterniones 

In [1]:
import keras
import numpy as np

Using TensorFlow backend.


La base de datos debe estar compuesta por  la diferencia de cuaterniones (la primera)

In [2]:
database = open("dataset2train","r") 
proteins = database.readlines()
database.close()

In [3]:
from random import shuffle
shuffle(proteins)

In [4]:
print("Número de proteínas: ",len(proteins))

lim = int(len(proteins)*0.4)
print("Proteinas en test: ",lim)
print("Proteinas en train: ",len(proteins)-lim)

Número de proteínas:  3275
Proteinas en test:  1310
Proteinas en train:  1965


In [5]:
maxlen = 15
step = 3
sequences_before_train = []
sequences_after_train = []
residues_train = []
next_chars_train = []
quaternions_before_train = []
quaternions_after_train = []


for protein in proteins[lim:]:
  
    nres = int((len(protein.split())+4)/5)
    sequence = protein.split()[0:nres]
    quaternion = protein.split()[nres:]

    
    for i in range(0, len(sequence) - maxlen + 1, step):
       
        sequences_before_train.append(sequence[i: i + int(maxlen/2)])
        sequences_after_train.append(sequence[i + int(maxlen/2)+1: i + maxlen])
        quat_before_train = quaternion[4*i: 4*(i - 1) + 4*int(maxlen/2)]
        quat_after_train = quaternion[4*(i - 1) + 4*int(maxlen/2+1): 4*(i - 2) + 4*(maxlen)]
        
        quaternions_before_train.append(quat_before_train)
        quaternions_after_train.append(quat_after_train)

        residues_train.append(sequence[i + int(maxlen/2)])

        next_chars_train.append(quaternion[4*(i - 1) + 4*int(maxlen/2):4*(i - 1) + 4*int(maxlen/2+1)])

In [6]:
len_to_take = int(maxlen/2)

In [7]:
maxlen = 15
step = 3
sequences_before_test = []
sequences_after_test = []
residues_test = []
next_chars_test = []
quaternions_before_test = []
quaternions_after_test = []

for protein in proteins[0:lim]:
    nres = int((len(protein.split())+4)/5)
    sequence = protein.split()[0:nres]
    quaternion = protein.split()[nres:]

    for i in range(0, len(sequence) - maxlen + 1, step):
        sequences_before_test.append(sequence[i: i + int(maxlen/2)])
        sequences_after_test.append(sequence[i + int(maxlen/2)+1: i + maxlen])
        quat_before_test = quaternion[4*i: 4*(i - 1) + 4*int(maxlen/2)]
        quat_after_test = quaternion[4*(i - 1) + 4*int(maxlen/2+1): 4*(i - 2) + 4*(maxlen)]

        quaternions_before_test.append(quat_before_test)
        quaternions_after_test.append(quat_after_test)

        residues_test.append(sequence[i + int(maxlen/2)])
        
        next_chars_test.append(quaternion[4*(i - 1) + 4*int(maxlen/2):4*(i - 1) + 4*int(maxlen/2+1)])

In [8]:
flatten = lambda l: [item for  sublist in l for item in sublist]
flatten_sbta = flatten(sequences_before_train)
flatten_sata = flatten(sequences_after_train)
flatten_sbte = flatten(sequences_before_test)
flatten_sate = flatten(sequences_after_test)

In [9]:
print('Number of sequences:', len(sequences_before_test+sequences_after_test))
chars = sorted(list(set(flatten_sbta + flatten_sata + flatten_sbte + flatten_sate)))
print('Unique characters:', len(chars))
print(chars)
char_indices = dict((char, chars.index(char)) for char in chars)

Number of sequences: 168572
Unique characters: 20
['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']


In [10]:
print('Vectorization...')
x_sequences_before_train = np.zeros((len(sequences_before_train), int(maxlen/2), len(chars)))
x_sequences_after_train = np.zeros((len(sequences_after_train), int(maxlen/2), len(chars)))
x_residues_train = np.zeros((len(sequences_before_train), 1, len(chars)))
x_quaternions_before_train = np.zeros((len(sequences_before_train), int(maxlen/2)-1, 4))
x_quaternions_after_train = np.zeros((len(sequences_after_train), int(maxlen/2)-1, 4))
y_train = np.zeros((len(sequences_before_train), 4))

for i, sequence in enumerate(sequences_before_train):
    for t, char in enumerate(sequence):
        x_sequences_before_train[i, t, char_indices[char]] = 1
        if(t < len(sequence) - 2):   
            
            x_quaternions_before_train[i, t, 0] = quaternions_before_train[i][4*t]
            x_quaternions_before_train[i, t, 1] = quaternions_before_train[i][4*t+1]
            x_quaternions_before_train[i, t, 2] = quaternions_before_train[i][4*t+2]
            x_quaternions_before_train[i, t, 3] = quaternions_before_train[i][4*t+3]

for i, sequence in enumerate(sequences_after_train):
    for t, char in enumerate(sequence):
        x_sequences_after_train[i, t, char_indices[char]] = 1
        if(t < len(sequence) - 2):
            x_quaternions_after_train[i, t, 0] = quaternions_after_train[i][4*t]
            x_quaternions_after_train[i, t, 1] = quaternions_after_train[i][4*t+1]
            x_quaternions_after_train[i, t, 2] = quaternions_after_train[i][4*t+2]
            x_quaternions_after_train[i, t, 3] = quaternions_after_train[i][4*t+3]
        
    x_residues_train[i, 0, char_indices[residues_train[i]]] = 1        
    y_train[i, 0] = next_chars_train[i][0]
    y_train[i, 1] = next_chars_train[i][1]
    y_train[i, 2] = next_chars_train[i][2]
    y_train[i, 3] = next_chars_train[i][3]

Vectorization...


In [11]:
print(x_sequences_before_train.shape)

(122981, 7, 20)


In [12]:
print('Vectorization...')
x_sequences_before_test = np.zeros((len(sequences_before_test), int(maxlen/2), len(chars)))
x_sequences_after_test = np.zeros((len(sequences_after_test), int(maxlen/2), len(chars)))
x_residues_test = np.zeros((len(sequences_before_test), 1, len(chars)))
x_quaternions_before_test = np.zeros((len(sequences_before_test), int(maxlen/2)-1, 4))
x_quaternions_after_test = np.zeros((len(sequences_after_test), int(maxlen/2)-1, 4))
y_test = np.zeros((len(sequences_before_test), 4))

for i, sequence in enumerate(sequences_before_test):
    for t, char in enumerate(sequence):
        x_sequences_before_test[i, t, char_indices[char]] = 1
        if(t < len(sequence) - 2):
            x_quaternions_before_test[i, t, 0] = quaternions_before_test[i][4*t]
            x_quaternions_before_test[i, t, 1] = quaternions_before_test[i][4*t+1]
            x_quaternions_before_test[i, t, 2] = quaternions_before_test[i][4*t+2]
            x_quaternions_before_test[i, t, 3] = quaternions_before_test[i][4*t+3]

for i, sequence in enumerate(sequences_after_test):
    for t, char in enumerate(sequence):
        x_sequences_after_test[i, t, char_indices[char]] = 1
        if(t < len(sequence) - 2):
            x_quaternions_after_test[i, t, 0] = quaternions_after_test[i][4*t]
            x_quaternions_after_test[i, t, 1] = quaternions_after_test[i][4*t+1]
            x_quaternions_after_test[i, t, 2] = quaternions_after_test[i][4*t+2]
            x_quaternions_after_test[i, t, 3] = quaternions_after_test[i][4*t+3]
        
    x_residues_test[i, 0, char_indices[residues_test[i]]] = 1        
    y_test[i, 0] = next_chars_test[i][0]
    y_test[i, 1] = next_chars_test[i][1]
    y_test[i, 2] = next_chars_test[i][2]
    y_test[i, 3] = next_chars_test[i][3]

Vectorization...


In [13]:
from keras import regularizers
from keras import Input, layers

input_sequences_before = Input(shape=(int(maxlen/2), len(chars)))
sequences_layers_before = layers.SimpleRNN(128,return_sequences=True,activation='relu')(input_sequences_before)
sequences_layers_before = layers.SimpleRNN(128,return_sequences=True,activation='relu')(sequences_layers_before)
sequences_layers_before = layers.SimpleRNN(128,activation='relu')(sequences_layers_before)
sequences_layers_before = layers.Dense(128,activation='relu')(sequences_layers_before)
sequences_layers_before = layers.Dropout(0.3)(sequences_layers_before)
sequences_layers_before = layers.Dense(128,activation='relu')(sequences_layers_before)
#output = layers.Dense(4, activation='linear')(sequences_layers_before)

W0130 16:50:54.306049 140378957707072 deprecation_wrapper.py:119] From /home/nef/miniconda3/envs/my_project/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0130 16:50:54.498802 140378957707072 deprecation_wrapper.py:119] From /home/nef/miniconda3/envs/my_project/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0130 16:50:54.533192 140378957707072 deprecation_wrapper.py:119] From /home/nef/miniconda3/envs/my_project/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0130 16:50:55.212398 140378957707072 deprecation_wrapper.py:119] From /home/nef/miniconda3/envs/my_project/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_

In [14]:
print(int(maxlen/2), len(chars))


7 20


In [15]:
from keras import Input, layers

input_sequences_after = Input(shape=(int(maxlen/2), len(chars)))
sequences_layers_after = layers.SimpleRNN(250,return_sequences=True,activation='relu')(input_sequences_after)
sequences_layers_after = layers.SimpleRNN(250,return_sequences=True,activation='relu')(sequences_layers_after)
sequences_layers_after = layers.SimpleRNN(250,activation='relu')(sequences_layers_after)
sequences_layers_after = layers.Dense(250,activation='relu')(sequences_layers_after)
sequences_layers_after = layers.Dense(250,activation='relu')(sequences_layers_after)
sequences_layers_after = layers.Dropout(0.001)(sequences_layers_after)
sequences_layers_after = layers.Dense(250,activation='relu')(sequences_layers_after)
#sequences_layers_after = layers.Dense(250,activation='relu')(sequences_layers_after)

In [16]:
from keras import Input, layers

input_residues = Input(shape=(1, len(chars)))
residues_layers = layers.SimpleRNN(250,return_sequences=True,activation='relu')(input_residues)
residues_layers = layers.SimpleRNN(250,return_sequences=True,activation='relu')(residues_layers)
residues_layers = layers.SimpleRNN(250,activation='relu')(residues_layers)
residues_layers = layers.Dense(250,activation='relu')(residues_layers)
residues_layers = layers.Dense(250,activation='relu')(residues_layers)
residues_layers = layers.Dropout(0.001)(residues_layers)
residues_layers = layers.Dense(250,activation='relu')(residues_layers)
#residues_layers = layers.Dense(250,activation='relu')(residues_layers)

In [17]:
input_quaternions_before = Input(shape=(int(maxlen/2-1), 4))
quaternions_layers_before = layers.SimpleRNN(250,return_sequences=True,activation='relu')(input_quaternions_before)
quaternions_layers_before = layers.SimpleRNN(250,return_sequences=True,activation='relu')(quaternions_layers_before)
quaternions_layers_before = layers.SimpleRNN(250,activation='relu')(quaternions_layers_before)#quaternions_layers_before = layers.Dense(250,activation='relu')(quaternions_layers_before)
quaternions_layers_before = layers.Dense(250,activation='relu')(quaternions_layers_before)
quaternions_layers_before = layers.Dropout(0.001)(quaternions_layers_before)
quaternions_layers_before = layers.Dense(250,activation='relu')(quaternions_layers_before)

In [18]:
print(1, len(chars))

1 20


In [19]:
313/60

5.216666666666667

In [20]:
from keras import regularizers
concatenated = layers.concatenate([sequences_layers_before, sequences_layers_after, residues_layers,quaternions_layers_before],axis = -1)
concatenated = layers.Dense(2000, activation='relu')(concatenated)
concatenated = layers.Dense(1000, activation='relu')(concatenated)
concatenated = layers.Dense(600, activation='relu')(concatenated)
concatenated = layers.Dense(500, activation='relu')(concatenated)
concatenated = layers.Dense(400, activation='relu')(concatenated)
concatenated = layers.Dropout(0.001)(concatenated)
concatenated = layers.Dense(300, activation='relu')(concatenated)
concatenated = layers.Dense(250, activation='relu')(concatenated)
output = layers.Dense(4, activation='linear')(concatenated)

In [21]:
from keras.models import Model

model = Model([input_sequences_before,input_sequences_after,input_residues,input_quaternions_before],output)

In [22]:
model.compile(loss='mae', optimizer='Adam')

W0130 16:50:57.025796 140378957707072 deprecation_wrapper.py:119] From /home/nef/miniconda3/envs/my_project/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [ ]:
history = model.fit([x_sequences_before_train,x_sequences_after_train,x_residues_train,x_quaternions_before_train], y_train, validation_data=([x_sequences_before_test,x_sequences_after_test,x_residues_test,x_quaternions_before_test],y_test), batch_size=100, epochs=65)

W0130 16:50:58.857059 140378957707072 deprecation_wrapper.py:119] From /home/nef/miniconda3/envs/my_project/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 122981 samples, validate on 84286 samples
Epoch 1/65
122981/122981 [==============================] - 299s 2ms/step - loss: 0.3244 - val_loss: 0.3096
Epoch 2/65
122981/122981 [==============================] - 285s 2ms/step - loss: 0.3084 - val_loss: 0.3033
Epoch 3/65
122981/122981 [==============================] - 286s 2ms/step - loss: 0.3033 - val_loss: 0.2958
Epoch 4/65
122981/122981 [==============================] - 286s 2ms/step - loss: 0.2920 - val_loss: 0.2884
Epoch 5/65
122981/122981 [==============================] - 297s 2ms/step - loss: 0.2871 - val_loss: 0.2854
Epoch 6/65
122981/122981 [==============================] - 298s 2ms/step - loss: 0.2829 - val_loss: 0.2832
Epoch 7/65
122981/122981 [==============================] - 298s 2ms/step - loss: 0.2784 - val_loss: 0.2778
Epoch 8/65
122981/122981 [==============================] - 300s 2ms/step - loss: 0.2752 - val_loss: 0.2768
Epoch 9/65
122981/122981 [==============================] - 299s 2ms/step - loss: 0.2

In [ ]:
from keras.models import load_model

model.save('fifty_epochs.h5') 


# Stop

In [ ]:
#from keras.models import load_model


#model = load_model('my_model.h5')

In [ ]:
from matplotlib import pyplot as plt
print(history.history.keys())
plt.plot( history.history['loss'],label='Train')
plt.plot(history.history['val_loss'],label='Test')
plt.xlabel('epochs')
plt.ylabel('mae')

plt.legend()
plt.savefig('learning_curve_original.png', dpi=400)
plt.legend()

In [ ]:
predictions = model.predict([x_sequences_before_test,x_sequences_after_test,x_residues_test,x_quaternions_before_test])

Probamos que  tan buena es la red 

In [ ]:
i = 1
for i in range(5):
    print("Original: ",y_test[i]," Prediccion: ",predictions[i])

# Validation


In [ ]:
import pyrr as pyrr
import math as mt
import numpy as np

database = open("validation_set.txt","r") 
proteins_2predict = database.readlines()
database.close()
proteins_2predict = proteins_2predict[:] #ojo aqui

dataset = open("validation_keys","r") 
claves_2predict = dataset.readlines()
dataset.close()


for line  in claves_2predict:
    keys = line.split(", ")
    b = keys[len(keys)-1].rstrip('\n')
    keys[len(keys)-1] = b
    keys = keys[:]

In [ ]:
print(keys)

In [ ]:
keys[2]

In [ ]:
keys[14]

In [ ]:
keys[16]

In [ ]:
keys[85]

In [ ]:
proteins_2predict.pop(85)

In [ ]:
proteins_2predict.pop(16)

In [ ]:
proteins_2predict.pop(14)

In [ ]:
proteins_2predict.pop(2)

In [ ]:
print(len(proteins_2predict))

In [ ]:
from angles import normalize

In [ ]:
def get_Ramachandran_straightness(Psi,Phi):
      
    theta= []
    i=1
    while i < len (Psi):
        angle=(Phi[i+1] + Psi[i] - Phi[i] - Psi[i-1])
        theta.append(normalize(angle, -180, 180))
        if i == (len(Psi)-2):
            break
        else:
            i=i+1
           
    s_ramachandran = []
    for i in range(len(theta)):
        s = 1-(abs(theta[i])/180)
        s_ramachandran.append(s)
    
    return s_ramachandran  

In [ ]:

def get_quaternions_straightness(x): #La  función toma las predicciones normalizadas de los quaterniones
    s_quaternions_diff_jmol=[]       #
    
    for i in range(len(x)-1):
        dot= pyrr.quaternion.dot(x[i+1], x[i])
        if dot>1:
            dot=1
        else:
            s = 1- (mt.acos(abs(dot))/(mt.pi/2))
   
            s_quaternions_diff_jmol.append(s)
        
    return s_quaternions_diff_jmol 

In [ ]:
for index, protein in enumerate(proteins_2predict):


    print("Proteina:",keys[index], "indice", index)
    maxlen = 15
    step = 1

    sequences_before_validation = []
    sequences_after_validation = []
    residues_validation = []
    quaternions_before_validation = []


    nres_val = int((len(protein.split())+4)/5)
    sequence_val = protein.split()[0:nres_val]
    quaternion = protein.split()[nres_val:]

    for i in range(0, len(sequence_val) - maxlen + 1, step):
        sequences_before_validation.append(sequence_val[i: i + int(maxlen/2)])    
        sequences_after_validation.append(sequence_val[i + int(maxlen/2)+1: i + maxlen])
        residues_validation.append(sequence_val[i + int(maxlen/2)])
        quat_before_v = quaternion[4*i: 4*(i - 1) + 4*int(maxlen/2)]
        quaternions_before_validation.append(quat_before_v)
    
    
    aa_2predict = sequence_val[7:len(sequence_val)-7]
    #a = "aa_2predict_of"
    #b = keys[index]
    #aa_2predict = open(a+b, "w")
    #aa_2predict.write(str(sequence_val[7:len(sequence_val)-7]))
    #aa_2predict.close
        
    print('Vectorization...')
    x_sequences_before_validation = np.zeros((len(sequences_before_validation), int(maxlen/2), len(chars)))
    x_sequences_after_validation = np.zeros((len(sequences_after_test), int(maxlen/2), len(chars)))
    x_residues_validation = np.zeros((len(sequences_before_validation), 1, len(chars)))
    x_quaternions_before_validation = np.zeros((len(sequences_before_validation), int(maxlen/2)-1, 4))
    y_test_validation = np.zeros((len(sequences_before_validation), 4))
    
#_________________________________________________________________________________________________________

    for i, sequence in enumerate(sequences_before_validation):
        for t, char in enumerate(sequence):
            x_sequences_before_validation[i, t, char_indices[char]] = 1
            if(t < len(sequence) - 2):
                x_quaternions_before_validation[i, t, 0] = quaternions_before_validation[i][4*t]
                x_quaternions_before_validation[i, t, 1] = quaternions_before_validation[i][4*t+1]
                x_quaternions_before_validation[i, t, 2] = quaternions_before_validation[i][4*t+2]
                x_quaternions_before_validation[i, t, 3] = quaternions_before_validation[i][4*t+3]

        
        x_residues_validation[i, 0, char_indices[residues_validation[i]]] = 1        
        y_test_validation[i, 0] = next_chars_test[i][0]
        y_test_validation[i, 1] = next_chars_test[i][1]
        y_test_validation[i, 2] = next_chars_test[i][2]
        y_test_validation[i, 3] = next_chars_test[i][3]
    
    print('Vectorization...')
    x_sequences_before_validation = np.zeros((len(sequences_before_validation), int(maxlen/2), len(chars)))
    x_sequences_after_validation = np.zeros((len(sequences_after_validation), int(maxlen/2), len(chars)))
    x_residues_validation = np.zeros((len(sequences_before_validation), 1, len(chars)))
    x_quaternions_before_validation = np.zeros((len(sequences_before_validation), int(maxlen/2)-1, 4))


    for i, sequence in enumerate(sequences_before_validation):
        for t, char in enumerate(sequence):
            x_sequences_before_validation[i, t, char_indices[char]] = 1
            if(t < len(sequence) - 2):
                x_quaternions_before_validation[i, t, 0] = quaternions_before_validation[i][4*t]
                x_quaternions_before_validation[i, t, 1] = quaternions_before_validation[i][4*t+1]
                x_quaternions_before_validation[i, t, 2] = quaternions_before_validation[i][4*t+2]
                x_quaternions_before_validation[i, t, 3] = quaternions_before_validation[i][4*t+3]

    for i, sequence in enumerate(sequences_after_validation):
        for t, char in enumerate(sequence):
            x_sequences_after_validation[i, t, char_indices[char]] = 1        
        x_residues_validation[i, 0, char_indices[residues_validation[i]]] = 1
    
#_____________________________________________________________________________________________________

    print("generando las predicciones...")   
    predictions_validation = model.predict([x_sequences_before_validation,x_sequences_after_validation,x_residues_validation,x_quaternions_before_validation])
#_____________________________________________________________________________________________________

    print("Normalizando las predicciones...")
    pred_normalized=[]
    for i in  range(len(predictions_validation)):
        pred_normalized.append(pyrr.quaternion.normalise(predictions_validation[i]))
        
    predictions_normalized=[]
    for j in range(len(pred_normalized)):
        cuarteto=[]
        cuarteto.append(pred_normalized[j][0])
        cuarteto.append(pred_normalized[j][1])
        cuarteto.append(pred_normalized[j][2])
        cuarteto.append(pred_normalized[j][3])
        predictions_normalized.append(cuarteto)
        
#______________________________________________________________________________________________________


    
    key_lower = keys[index].lower()
    print(key_lower, keys[index]) 
#________________________________________________________________________________________________________

    from prody import *
    from pylab import *
    from pyrr import Quaternion, Matrix33, Matrix44, Vector3, Vector4
    import py3Dmol

    %matplotlib inline
    confProDy(auto_show=False)
    confProDy(auto_secondary=True)


    p38 = parsePDB(key_lower)

#______________________________________________________________________________________________________

    print("generando el  straightness de Ramachandran para la cadena A")
          
    #para la cadena A (arreglar para aceptar mas cadenas)      
          
    p38['A'].getSequence()          
    chain = p38['A']
    Phi = []; Psi = []; c = []
    for res in chain.iterResidues():
        try:
            phi = calcPhi(res)
            psi = calcPsi(res)
        except:
            continue
        else:
            Phi.append(phi)
            Psi.append(psi)
            if res.getResname() == 'GLY':
                c.append('r')
            else:
                c.append('b')   
                
#_________________________________________________________________________________________________________

    amino_number= len(Psi)+2
    amino_number
#_____________________________________________________________________________________________________

    ppsi = Psi[len_to_take-2:(amino_number-len_to_take)-1]
    
#_____________________________________________________________________________________________________

    pphi = Phi[len_to_take-2:(amino_number-len_to_take)-1]
    
#_____________________________________________________________________________________________________


    ramachandran_straightness = get_Ramachandran_straightness(ppsi,pphi)
    print("len(ramachandran_straightness)", len(ramachandran_straightness))
#____________________________________________________________________________________________________


    straightnes_quaternions = get_quaternions_straightness(predictions_normalized)
    print("len(straightnes_quaternions)", len(straightnes_quaternions))
        
#____________________________________________________________________________________________________

    aa_2predict.pop()
#____________________________________________________________________________________________________

    print("No. de aminoácidos a predecir ", len( aa_2predict))
    
#____________________________________________________________________________________________________

    import matplotlib.pyplot as plt 
    %matplotlib notebook
    plt.plot(ramachandran_straightness, straightnes_quaternions, 'ro')
    plt.axis([0, 1,0, 1])
    plt.ylabel('Ramachandran-C-Based-Straightness')
    plt.xlabel('quaternion-C-Based-Straightness')
    extension = '.png'
    plt.savefig(key_lower + extension, dpi=400)
    
    
#_____________________________________________________________________________________________________

    from scipy import stats
    import numpy as np
    slope, intercept, r_value, p_value, std_err = stats.linregress(ramachandran_straightness, straightnes_quaternions)

#____________________________________________________________________________________________________

    print ("r-squared:", r_value**2)
    
#_____________________________________________________________________________________________________

    print("slop", slope)
    plt.show()
    
